In [4]:
%%writefile mpi_mean_std.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <cstdlib>

using namespace std;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int N = 1'000'000;

    vector<double> data;
    vector<int> sendCounts(size), displs(size);

    // Только процесс 0 создаёт массив
    if (rank == 0) {
        data.resize(N);
        srand(42);
        for (int i = 0; i < N; i++) {
            data[i] = static_cast<double>(rand()) / RAND_MAX;
        }

        int base = N / size;
        int remainder = N % size;

        int offset = 0;
        for (int i = 0; i < size; i++) {
            sendCounts[i] = base + (i < remainder ? 1 : 0);
            displs[i] = offset;
            offset += sendCounts[i];
        }
    }

    // Рассылка размеров частей
    int localSize;
    MPI_Scatter(sendCounts.data(), 1, MPI_INT,
                &localSize, 1, MPI_INT,
                0, MPI_COMM_WORLD);

    vector<double> localData(localSize);

    // Распределение массива
    MPI_Scatterv(data.data(), sendCounts.data(), displs.data(), MPI_DOUBLE,
                 localData.data(), localSize, MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // Локальные вычисления
    double localSum = 0.0;
    double localSumSquares = 0.0;

    for (double x : localData) {
        localSum += x;
        localSumSquares += x * x;
    }

    // Глобальные суммы
    double globalSum = 0.0;
    double globalSumSquares = 0.0;

    MPI_Reduce(&localSum, &globalSum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&localSumSquares, &globalSumSquares, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    // Вычисление результата
    if (rank == 0) {
        double mean = globalSum / N;
        double stddev = sqrt(globalSumSquares / N - mean * mean);

        cout << "Среднее значение: " << mean << endl;
        cout << "Стандартное отклонение: " << stddev << endl;
    }

    MPI_Finalize();
    return 0;
}


Overwriting mpi_mean_std.cpp


In [7]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mean_std



Среднее значение: 0.500147
Стандартное отклонение: 0.288672


In [8]:
%%writefile mpi_gauss.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <cmath>

using namespace std;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Размер системы (можно менять)
    const int N = 4;

    vector<double> A;
    vector<double> b;

    int rowsPerProc = N / size;

    vector<double> localA(rowsPerProc * N);
    vector<double> localB(rowsPerProc);

    // Инициализация только на rank = 0
    if (rank == 0) {
        A = {
            10, 2, 3, 4,
             3,10, 4, 5,
             1, 2,10, 6,
             2, 3, 4,10
        };

        b = {16, 22, 19, 29};
    }

    // Распределение строк матрицы и вектора
    MPI_Scatter(A.data(), rowsPerProc * N, MPI_DOUBLE,
                localA.data(), rowsPerProc * N, MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    MPI_Scatter(b.data(), rowsPerProc, MPI_DOUBLE,
                localB.data(), rowsPerProc, MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    vector<double> pivotRow(N);
    double pivotB;

    // Прямой ход
    for (int k = 0; k < N; k++) {
        int owner = k / rowsPerProc;

        if (rank == owner) {
            int localRow = k % rowsPerProc;
            for (int j = 0; j < N; j++) {
                pivotRow[j] = localA[localRow * N + j];
            }
            pivotB = localB[localRow];
        }

        MPI_Bcast(pivotRow.data(), N, MPI_DOUBLE, owner, MPI_COMM_WORLD);
        MPI_Bcast(&pivotB, 1, MPI_DOUBLE, owner, MPI_COMM_WORLD);

        for (int i = 0; i < rowsPerProc; i++) {
            int globalRow = rank * rowsPerProc + i;
            if (globalRow > k) {
                double factor = localA[i * N + k] / pivotRow[k];
                for (int j = k; j < N; j++) {
                    localA[i * N + j] -= factor * pivotRow[j];
                }
                localB[i] -= factor * pivotB;
            }
        }
    }

    // Сбор матрицы и вектора на rank = 0
    MPI_Gather(localA.data(), rowsPerProc * N, MPI_DOUBLE,
               A.data(), rowsPerProc * N, MPI_DOUBLE,
               0, MPI_COMM_WORLD);

    MPI_Gather(localB.data(), rowsPerProc, MPI_DOUBLE,
               b.data(), rowsPerProc, MPI_DOUBLE,
               0, MPI_COMM_WORLD);

    // Обратный ход (только rank = 0)
    if (rank == 0) {
        vector<double> x(N);

        for (int i = N - 1; i >= 0; i--) {
            x[i] = b[i];
            for (int j = i + 1; j < N; j++) {
                x[i] -= A[i * N + j] * x[j];
            }
            x[i] /= A[i * N + i];
        }

        cout << "Решение системы:" << endl;
        for (int i = 0; i < N; i++) {
            cout << "x[" << i << "] = " << x[i] << endl;
        }
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_gauss.cpp


In [9]:
!mpic++ mpi_gauss.cpp -O2 -o mpi_gauss
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_gauss


Решение системы:
x[0] = 0.382895
x[1] = 0.744966
x[2] = 0.200998
x[3] = 2.51953


In [10]:
%%writefile mpi_floyd.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <algorithm>

using namespace std;

const int INF = 1000000000;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Размер графа (можно менять)
    const int N = 4;

    vector<int> G;
    vector<int> localG;

    int rowsPerProc = N / size;

    // Инициализация графа только на rank = 0
    if (rank == 0) {
        G = {
            0,   3,   INF, 7,
            8,   0,   2,   INF,
            5,   INF, 0,   1,
            2,   INF, INF, 0
        };
    }

    localG.resize(rowsPerProc * N);

    // Распределение строк матрицы
    MPI_Scatter(G.data(), rowsPerProc * N, MPI_INT,
                localG.data(), rowsPerProc * N, MPI_INT,
                0, MPI_COMM_WORLD);

    vector<int> fullG(N * N);

    // Алгоритм Флойда–Уоршелла
    for (int k = 0; k < N; k++) {
        // Собираем обновлённую матрицу у всех процессов
        MPI_Allgather(localG.data(), rowsPerProc * N, MPI_INT,
                      fullG.data(), rowsPerProc * N, MPI_INT,
                      MPI_COMM_WORLD);

        // Обновление локальных строк
        for (int i = 0; i < rowsPerProc; i++) {
            int globalI = rank * rowsPerProc + i;
            for (int j = 0; j < N; j++) {
                int throughK = fullG[globalI * N + k] + fullG[k * N + j];
                localG[i * N + j] = min(localG[i * N + j], throughK);
            }
        }
    }

    // Сбор финальной матрицы на rank = 0
    MPI_Gather(localG.data(), rowsPerProc * N, MPI_INT,
               G.data(), rowsPerProc * N, MPI_INT,
               0, MPI_COMM_WORLD);

    // Вывод результата
    if (rank == 0) {
        cout << "Матрица кратчайших расстояний:" << endl;
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                if (G[i * N + j] >= INF)
                    cout << "INF ";
                else
                    cout << G[i * N + j] << " ";
            }
            cout << endl;
        }
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_floyd.cpp


In [11]:
!mpic++ mpi_floyd.cpp -O2 -o mpi_floyd
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_floyd


Матрица кратчайших расстояний:
0 3 5 6 
5 0 2 3 
3 6 0 1 
2 5 7 0 
